In [31]:
import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError


# Database Connections 

In [32]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['fuente']
    config_etl = config['bodega']
# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
ra_fu = create_engine(url_co)
etl_conn = create_engine(url_etl)

In [33]:

try:
    # Probar la conexión a la base de datos de origen
    with ra_fu.connect() as connection:
        print("Conexión a la base de datos de origen exitosa.")
        
    # Probar la conexión a la base de datos de destino
    with etl_conn.connect() as connection:
        print("Conexión a la base de datos de destino exitosa.")

except OperationalError as e:
    print("Error al conectarse a la base de datos:", e)


Conexión a la base de datos de origen exitosa.
Conexión a la base de datos de destino exitosa.


# Extract

In [34]:
# Muestra las tablas en la base de datos de origen
ra_fu.table_names()


C:\Users\Usuario\AppData\Local\Temp\ipykernel_20364\3220893687.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  ra_fu.table_names()


['admin_interface_theme',
 'areas_cliente',
 'auth_user_user_permissions',
 'clientes_coordinador',
 'clientes_datosmensajero',
 'clientes_mensajeroaquitoy_clientes',
 'auth_permission',
 'auth_group',
 'auth_user_groups',
 'auth_user',
 'authtoken_token',
 'ciudad',
 'cliente',
 'clientes_mensajeroaquitoy',
 'area',
 'django_migrations',
 'departamento',
 'clientes_usuarioaquitoy',
 'django_admin_log',
 'django_session',
 'mensajeria_novedadesservicio',
 'mensajeria_tiponovedad',
 'mensajeria_origenservicio',
 'mensajeria_tipopago',
 'mensajeria_estado',
 'mensajeria_tiposervicio',
 'mensajeria_servicio',
 'mensajeria_estadosservicio',
 'mensajeria_tipovehiculo',
 'sede',
 'auth_group_permissions',
 'django_content_type',
 'tipo_cliente',
 'mensajeria_destinoservicio',
 'mensajeria_documentoasociado']

In [35]:
try:
    # Crear una conexión activa
    with ra_fu.connect() as conn:
        query = "SELECT * FROM cliente"  # Cambia 'cliente' por el nombre de tu tabla de origen
        dim_cliente = pd.read_sql(query, conn)  # Ejecutar la consulta en la conexión activa
        print(dim_cliente.head())  # Mostrar los primeros registros de la tabla cargada
except Exception as e:
    print("Error al cargar los datos de la tabla:", e)


Error al cargar los datos de la tabla: 'Connection' object has no attribute 'cursor'


C:\Users\Usuario\AppData\Local\Temp\ipykernel_20364\2232479241.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_cliente = pd.read_sql(query, conn)  # Ejecutar la consulta en la conexión activa


In [29]:
import pandas as pd
import sqlalchemy
print("Pandas version:", pd.__version__)
print("SQLAlchemy version:", sqlalchemy.__version__)

Pandas version: 2.2.3
SQLAlchemy version: 1.4.54


# Transformations

In [ ]:
dim_cliente = pd.DataFrame({
    'descripcion' : ['descripcion'],
    'fecha' : ['fecha_solicitud']
})

In [ ]:
dim_cliente

,descripcion,fecha
0,descripcion,fecha_solicitud


In [4]:
from datetime import date

dim_servicio['saved'] = date.today()

# load

In [5]:
dim_servicio.head()

,name,descripcion,saved
0,citas,servicio de citas medicas,2024-09-17
1,hospitalizacion,servicio de hospitalizacion,2024-09-17
2,urgencias,servicio de urgencias,2024-09-17


In [6]:
dim_servicio.to_sql('dim_servicio', etl_conn, if_exists='replace',index_label='key_dim_servicio')

3